# Introduction - Hyperparameters Search

In this notebook we demonstrate the use of **BM25 (Best Matching 25)** Information Retrieval technique to make trace link recovery between Test Cases and Bug Reports.

We model our study as follows:

* Each bug report title, summary and description compose a single query.
* We use each test case content as an entire document that must be returned to the query made


In [1]:
from mod_finder_util import mod_finder_util
mod_finder_util.add_modules_origin_search_path()

import pandas as pd
import numpy as np

from sklearn.externals.joblib import Parallel, delayed

from modules.utils import plots
from modules.utils import firefox_dataset_p2 as fd
from modules.utils import tokenizers as tok
from modules.utils import aux_functions
from modules.utils import model_evaluator as m_eval

from modules.models.bm25 import BM_25
from modules.models.model_hyperps import BM25_Model_Hyperp

import warnings; warnings.simplefilter('ignore')

## Load Dataset

In [2]:
test_cases_df = fd.Datasets.read_testcases_df()
bug_reports_df = fd.Datasets.read_selected_bug_reports_2_df()

corpus = test_cases_df.tc_desc
query = bug_reports_df.br_desc

test_cases_names = test_cases_df.tc_name
bug_reports_names = bug_reports_df.br_name

orc = fd.Tc_BR_Oracles.read_oracle_expert_volunteers_df()

TestCases.shape: (207, 12)
SelectedBugReports2.shape: (93, 22)
OracleExpertVolunteers.shape: (207, 93)


### Select Subset

In [12]:
bugreports_subset_df = bug_reports_df[(bug_reports_df.Version == '48 Branch') | (bug_reports_df.Version == '60 Branch')].sample(15, random_state=42)
testcases_subset_df = test_cases_df[(test_cases_df.TestDay.str.contains('20161014')) | (test_cases_df.TestDay.str.contains('20161028'))].sample(10, random_state=1000)

selected_testcases = ['TC_{}_TRG'.format(tc_num) for tc_num in [13, 14, 15, 16, 17, 18]]  # should link with 48 Branch
aux_tc = test_cases_df[test_cases_df.tc_name.isin(selected_testcases)]

tc_subset_df = testcases_subset_df.append(aux_tc)
tc_subset_df.drop_duplicates(inplace=True)

corpus_subset = tc_subset_df.tc_desc
query_subset = bugreports_subset_df.br_desc
testcases_names_subset = tc_subset_df.tc_name
bug_reports_names_subset = bugreports_subset_df.br_name
orc_subset_df = orc.loc[testcases_names_subset, bug_reports_names_subset]

print('TestCases Subset Shape: {}'.format(tc_subset_df.shape))
print('BugReports Subset Shape: {}'.format(bugreports_subset_df.shape))
print('Oracle Subset Shape: {}'.format(orc_subset_df.shape))

TestCases Subset Shape: (14, 10)
BugReports Subset Shape: (15, 12)
Oracle Subset Shape: (14, 15)


### Find The Best Model

In [13]:
all_hyperparams = {
    BM25_Model_Hyperp.TOP.value : [3,5],
    BM25_Model_Hyperp.SIM_MEASURE_MIN_THRESHOLD.value : [('-', 0.0)],
    BM25_Model_Hyperp.TOKENIZER.value : [tok.PorterStemmerBased_Tokenizer(), tok.LancasterStemmerBased_Tokenizer(), 
                                         tok.WordNetBased_LemmaTokenizer(), tok.SnowballStemmerBased_Tokenizer()]
}

hyperparams = aux_functions.generate_params_comb_list(**all_hyperparams)

print('Performing model hyperparameters search...')

def run_model(idx, **hyperp):    
    current_model = BM_25(**hyperp)
    current_model.set_name('BM25_Model_{}'.format(idx))
    current_model.recover_links(corpus_subset, query_subset, testcases_names_subset, bug_reports_names_subset)
    
    evaluator = m_eval.ModelEvaluator(orc_subset_df, current_model)
    evaluator.evaluate_model()
    evaluator.dump_model()
    evaluator.dump_evaluator()
    
    return([evaluator.get_mean_precision(), 
            evaluator.get_mean_recall(),
            evaluator.get_mean_fscore(), 
            evaluator.get_model().get_name(),
            evaluator.get_model().get_top_value(),
            evaluator.get_model().get_tokenizer_type(),
            evaluator.get_model().get_model_dump_path(),
            evaluator.get_evaluator_dump_path()
           ])

tasks = [(idx,hp) for idx,hp in enumerate(hyperparams)]
results = Parallel(n_jobs=-1, verbose=1)(delayed(run_model)(idx,**hp) for idx,hp in tasks)
results_df = pd.DataFrame(data=results, 
                          columns=['precision', 'recall', 'fscore', 'model_name', 'top_value', 'tokenizer', 'model_dump', 'evaluator_dump'])
results_df = results_df.astype(dtype={'model_dump' : str, 'evaluator_dump' : str})


Performing model hyperparameters search...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed:    0.2s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    1.4s finished


### Report

In [14]:
best_model = aux_functions.report_best_model(results_df)

------------ Report -------------------

Total of Analyzed Hyperparameters Combinations: 8

Best Model Hyperparameters Combination Found:

{'Measures': {'Mean FScore of BM25_Model_6': 0.13333333333333333,
              'Mean Precision of BM25_Model_6': 0.16,
              'Mean Recall of BM25_Model_6': 0.11428571428571428},
 'Setup': [{'Name': 'BM25_Model_6'},
           {'Top Value': 5},
           {'Sim Measure Min Threshold': ('-', 0.0)},
           {'K': 1.2},
           {'B': 0.75},
           {'Epsilon': 0.25},
           {'Tokenizer Type': <class 'utils.tokenizers.WordNetBased_LemmaTokenizer'>}]}


### Save Similarity Matrix

In [15]:
best_model.save_sim_matrix()

#### Best Model for TOP 3 and 5

In [16]:
aux_functions.print_report_top_3_and_5_v1(results_df)

{'Measures': {'Mean FScore of BM25_Model_2': 0.10666666666666667,
              'Mean Precision of BM25_Model_2': 0.17777777777777776,
              'Mean Recall of BM25_Model_2': 0.07619047619047618},
 'Setup': [{'Name': 'BM25_Model_2'},
           {'Top Value': 3},
           {'Sim Measure Min Threshold': ('-', 0.0)},
           {'K': 1.2},
           {'B': 0.75},
           {'Epsilon': 0.25},
           {'Tokenizer Type': <class 'utils.tokenizers.WordNetBased_LemmaTokenizer'>}]}
------------------------------------------------------------------
{'Measures': {'Mean FScore of BM25_Model_6': 0.13333333333333333,
              'Mean Precision of BM25_Model_6': 0.16,
              'Mean Recall of BM25_Model_6': 0.11428571428571428},
 'Setup': [{'Name': 'BM25_Model_6'},
           {'Top Value': 5},
           {'Sim Measure Min Threshold': ('-', 0.0)},
           {'K': 1.2},
           {'B': 0.75},
           {'Epsilon': 0.25},
           {'Tokenizer Type': <class 'utils.tokenizers.WordNet